## 1. 설정 & 전처리

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
import time 
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import numpy as np
from sklearn.cluster import DBSCAN
from xgboost import XGBClassifier, DMatrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.combine import SMOTETomek
import re

import random
random.seed(804)

In [38]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [39]:
# 변수 형태별 리스트
cat_list = [a for a in train.columns if a.endswith('cat')]
bin_list = [a for a in train.columns if a.endswith('bin')]
int_list = [a for a in train.columns if a not in cat_list and a not in bin_list]

In [40]:
# 범주형 변환 
train[cat_list] = train[cat_list].astype('category')
test[cat_list] = test[cat_list].astype('category')

In [10]:
#범주형은 결측치 대체X
train = train.replace(np.nan, 999)
test = test.replace(np.nan, 999)

In [41]:
vars_to_drop = ['V24_cat', 'V26_cat']
train.drop(vars_to_drop, inplace=True, axis=1)
test.drop(vars_to_drop, inplace=True, axis=1)

from statistics import mode

train.loc[train['V21'] != train['V21'], 'V21'] = train['V21'].mean()
train.loc[train['V34'] != train['V34'], 'V34'] = train['V34'].mean()
train.loc[train['V36'] != train['V36'], 'V36'] = train['V36'].mean()

train.loc[train['V5'] != train['V5'], 'V5'] = mode(train['V5'])
train.loc[train['V33'] != train['V33'], 'V33'] = mode(train['V33'])

test.loc[test['V21'] != test['V21'], 'V21'] = test['V21'].mean()
test.loc[test['V34'] != test['V34'], 'V34'] = test['V34'].mean()
test.loc[test['V36'] != test['V36'], 'V36'] = test['V36'].mean()

test.loc[test['V5'] != test['V5'], 'V5'] = mode(test['V5'])
test.loc[test['V33'] != test['V33'], 'V33'] = mode(test['V33'])



In [42]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # y변수 평균 구하기
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # smoothing 구하기
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
 
    prior = target.mean()
    # 카운트수 크면 full_avg 작게
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [43]:
#인코딩
train_encoded, test_encoded = target_encode(train["V32_cat"], 
                             test["V32_cat"], 
                             target=train.target, 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    
train['V32_cat_encoded'] = train_encoded
train.drop('V32_cat', axis=1, inplace=True)
test['V32_cat_encoded'] = test_encoded
test.drop('V32_cat', axis=1, inplace=True)

In [5]:
#언더샘플링
from sklearn.utils import shuffle

desired_apriori=0.10


idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index


nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])


undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('0 언더샘플링 비율: {}'.format(undersampling_rate))
print('0 counts: {}'.format(undersampled_nb_0))


undersampled_idx = shuffle(idx_0, random_state=314, n_samples=undersampled_nb_0)


idx_list = list(undersampled_idx) + list(idx_1)


train = train.loc[idx_list].reset_index(drop=True)

0 언더샘플링 비율: 0.341441237210994
0 counts: 137061


In [45]:
#랜덤 오버 샘플링으로 클래스 불균형 개선 
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)

print("sampling 적용 전 학습용 피처/레이블 데이터 세트 : ", X.shape, y.shape)
print('sampling 적용 후 학습용 피처/레이블 데이터 세트 :', X_over.shape, y_over.shape)

sampling 적용 전 학습용 피처/레이블 데이터 세트 :  (416648, 55) (416648,)
sampling 적용 후 학습용 피처/레이블 데이터 세트 : (802838, 55) (802838,)


In [26]:
cat_features = [a for a in train.columns if a.endswith('cat')]

for column in cat_features:
    temp = pd.get_dummies(pd.Series(train[column]))
    train = pd.concat([train,temp],axis=1)
    train = train.drop([column],axis=1)
    
for column in cat_features:
    temp = pd.get_dummies(pd.Series(test[column]))
    test = pd.concat([test,temp],axis=1)
    test = test.drop([column],axis=1)

In [44]:
id_test = test['id'].values
y = train['target'].values

X = train.drop(['target','id'], axis = 1)
test = test.drop(['id'], axis = 1)

In [231]:
print("Train dataset :",train.values.shape, "\nTest dataset:",test.values.shape)

Train dataset : (802838, 55) 
Test dataset: (178564, 55)



## 2. 모델링

In [103]:
train.shape[0]

152290

In [9]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    # X = train, y = target_train, T = test
    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)
        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=314).split(X, y))

        # 0으로만 채워진 array 생성
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))

        # n개 모델 돌려가면서 결과냄
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                print("Base model %d: fit %s model | fold %d" % (i+1, str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
                tr_preds = clf.predict(X_train)
                tr_score = f1_score(tr_preds, y_train)
                print("f1: ", tr_score)
                y_pred = clf.predict_proba(X_holdout)[:,1]               
                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        self.stacker.fit(S_train, y)
        results = self.stacker.predict(S_train)
        results_score = f1_score(results, y)
        print("Final f1: ", results_score)

        res = self.stacker.predict(S_test)
        return res

In [10]:
# LightGBM params
# lgb_1
lgb_params1 = {}
lgb_params1['learning_rate'] = 0.02
lgb_params1['n_estimators'] = 1000
lgb_params1['random_state'] = 804
lgb_params1['max_depth'] = 6
lgb_params1['min_child_samples'] = 500
lgb_params1['num_leaves'] = 64
lgb_params1['num_threads'] = 4
lgb_params1['is_unbalance'] = True



# lgb2
lgb_params2 = {}
lgb_params2['learning_rate'] = 0.02
lgb_params2['n_estimators'] = 600
lgb_params2['random_state'] = 804
lgb_params2['max_depth'] = 6
lgb_params2['min_child_samples'] = 500
lgb_params2['num_leaves'] = 64
lgb_params2['num_threads'] = 4
lgb_params1['is_unbalance'] = True


# lgb3
lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['seed'] = 314
lgb_params3['num_threads'] = 4

# XGBoost params
xgb_params = {}
xgb_params['objective'] = 'binary:logistic'
xgb_params['learning_rate'] = 0.04
xgb_params['n_estimators'] = 490
xgb_params['max_depth'] = 4
xgb_params['subsample'] = 0.9
xgb_params['colsample_bytree'] = 0.9  
xgb_params['min_child_weight'] = 10
xgb_params['num_threads'] = 4

In [11]:
lgb_model1 = LGBMClassifier(**lgb_params1)

lgb_model2 = LGBMClassifier(**lgb_params2)
       
lgb_model3 = LGBMClassifier(**lgb_params3)

xgb_model = XGBClassifier(**xgb_params)

# Stacking model
log_model = LogisticRegression()

In [14]:
stack = Ensemble(n_splits=2,
        stacker = log_model,
        base_models = (lgb_model1, xgb_model))  

In [ ]:
y_prediction = stack.fit_predict(train, target_train, test)

Base model 1: fit LGBMClassifier model | fold 1
[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be ignored. Current value: num_threads=4
f1:  0.33925427079793646
Base model 1: fit LGBMClassifier model | fold 2
[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be ignored. Current value: num_threads=4
f1:  0.3454306130124106
Base model 2: fit XGBClassifier model | fold 1
[22:47:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_threads" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




In [306]:
submission = pd.DataFrame()
submission['id'] = id_test
submission['target'] = y_prediction
submission.to_csv('1-10.csv', index=False)